In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d donkeys/retinopathy-train-2015 -p /content

import zipfile
with zipfile.ZipFile('/content/retinopathy-train-2015.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

print("Dataset downloaded and extracted to /content")


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/donkeys/retinopathy-train-2015
License(s): unknown
100% 33.8G/33.8G [04:38<00:00, 80.6MB/s]
100% 33.8G/33.8G [04:38<00:00, 130MB/s] 
Dataset downloaded and extracted to /content


In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.debug.metrics as met
import torch_xla.distributed.xla_multiprocessing as xmp
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset, random_split
import timm
import pandas as pd
from PIL import Image
import os

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0] + ".png")
        image = Image.open(img_path).convert("RGB")
        label = int(self.data.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

def trim_dataset(dataset, batch_size):
    trim_size = (len(dataset) // batch_size) * batch_size
    return Subset(dataset, range(trim_size))

batch_size = 64
val_split = 0.2 

full_dataset = DiabeticRetinopathyDataset("/content/trainLabels.csv", "/content/rescaled_train_896/rescaled_train_896", transform)
train_size = int((1 - val_split) * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataset = trim_dataset(train_dataset, batch_size)
val_dataset = trim_dataset(val_dataset, batch_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

def get_model():
    model = timm.create_model("seresnext50_32x4d", pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 5) 
    return model

def train(rank):
    device = xm.xla_device()
    model = get_model().to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        loader = pl.MpDeviceLoader(train_loader, device)
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            xm.optimizer_step(optimizer)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")

        xm.save(model.state_dict(), "dr_seresnext50_2015_tpu.pth")  # Save model

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        loader = pl.MpDeviceLoader(val_loader, device)
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Validation Accuracy: {100*correct/total:.2f}%")

if __name__ == "__main__":
    xmp.spawn(train, args=(), nprocs=1, start_method="fork")


/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model.safetensors:   0%|          | 0.00/111M [00:00<?, ?B/s]

Epoch 1/10, Loss: 0.7414, Accuracy: 75.42%
Epoch 2/10, Loss: 0.5541, Accuracy: 80.69%
Epoch 3/10, Loss: 0.3684, Accuracy: 86.53%
Epoch 4/10, Loss: 0.1830, Accuracy: 93.57%
Epoch 5/10, Loss: 0.0896, Accuracy: 96.92%
